# Econ 122 HW 1
### Oscar Scholin

In [29]:
# imports
import pyreadstat as prs
from helper import *

# read in list of files
# files = [f'data/morg79.dta'] + [f'data/morg{j}{l}.dta' for j in [8, 9, 0, 1] for l in range(0, 10)] + [f'data/morg20.dta']
files = ['data/morg79.dta','data/morg19.dta' ]

## Problem 1

In [56]:
# for each year, log hourly wage, % < high school, % with college, years edu, potential exp, potential exp squared, sample size
# do this for 3 criteria: men and women age 24-65, men age 24-65, women age 24-65
# weight based on hrearnwt = dataframe['earnwt'] * dataframe['uhours']

# define dictionary of dataframes
dfs = {}

for file in files:
    print(file)
    # create dataframe
    df, meta = prs.read_dta(file)
    # create year variable
    # split .dta, take last two characters, convert to int
    year = int(file.split('.')[0][-2:])
    dfs[year] = df

print(dfs.keys())

data/morg79.dta
data/morg19.dta
dict_keys([79, 19])


In [43]:
dfs[19].columns

Index(['hhid', 'intmonth', 'hurespli', 'hrhtype', 'minsamp', 'hrlonglk',
       'hrsample', 'hrhhid2', 'serial', 'hhnum', 'stfips', 'cbsafips',
       'county', 'centcity', 'smsastat', 'icntcity', 'smsa04', 'relref95',
       'age', 'spouse', 'sex', 'grade92', 'race', 'ethnic', 'lineno', 'famnum',
       'pfamrel', 'marital', 'prpertyp', 'penatvty', 'pemntvty', 'pefntvty',
       'prcitshp', 'prcitflg', 'peinusyr', 'selfproxy', 'lfsr94', 'absent94',
       'uhourse', 'reason94', 'hourslw', 'laydur', 'dwrsn', 'why3594',
       'prunedur', 'untype', 'ftpt94', 'class94', 'agri', 'eligible', 'otc',
       'ernpdh2', 'paidhre', 'earnhre', 'earnwke', 'unionmme', 'unioncov',
       'schenr', 'studftpt', 'schlvl', 'earnwt', 'weight', 'chldpres',
       'ownchild', 'I25d', 'I25c', 'I25a', 'I25b', 'qstnum', 'occurnum', 'ged',
       'gedhigr', 'yrcoll', 'grprof', 'gr6cor', 'ms123', 'cmpwgt', 'ind02',
       'occ2012', 'occ002', 'vet1', 'vet2', 'vet3', 'vet4', 'linedad',
       'linemom', 'recnum

In [57]:
# now we need to read in the inflation data and create df
# inflation = [33.804,34.728,35.590,36.451,37.545,38.464,39.364,40.338,41.384,42.078,42.772,43.427,43.979,44.402,45.103,45.601,45.978,46.398,47.008,47.318,47.830,48.293,48.666,48.966,49.544,49.945,50.337,50.688,51.047,50.993,51.263,51.573,52.061,52.564,53.062,53.520,53.943,54.539,55.211,55.769,56.410,57.170,57.507,57.959,58.799,59.333,60.087,60.882,61.202,61.536,61.954,62.404,62.795,63.212,63.615,64.059,64.441,64.874,65.155,65.531,65.766,66.133,66.607,66.920,67.247,67.639,67.915,68.214,68.593,69.052,69.346,69.819,70.127,70.303,70.488,70.710,70.715,70.843,71.062,71.249,71.390,71.796,72.191,72.629,73.219,73.568,74.042,74.460,75.012,75.363,75.401,75.432,75.584,76.145,76.539,76.896,77.484,77.562,78.073,78.457,79.062,79.594,79.985,80.669,81.138,81.650,82.532,83.189,83.621,84.355,84.962,84.822,85.597,86.324,86.812,87.694,88.408,89.269,90.221,88.781,88.181,88.532,89.142,89.830,90.177,90.317,90.490,91.070,91.835,92.738,93.167,93.475,94.094,94.321,94.595,95.126,95.459,95.508,95.901,96.254,96.696,97.129,97.394,97.265,96.829,97.312,97.565,97.490,97.538,98.157,98.496,98.947,99.524,99.724,100.076,100.676,101.380,101.912,102.254,102.642,102.877,103.422,103.674,104.080]
# # divide into groups of 4 for each quarter within a year
# inflation = [inflation[i:i+4] for i in range(0, len(inflation), 4)]
# inflation
inflation = [[33.804,34.728,35.590,36.451], [102.877, 103.422, 103.674, 104.080]]

In [59]:
# now that data is loaded in dictionary, can create tables
# add to each df the new variables
for j, year in enumerate(dfs.keys()):
    df = dfs[year]
    # create new variables

    # make sure to compare the right row entries so the division makes sense
    if 'uhours' in df.columns:
        # drop all values =< 0
        df.loc[df['uhours'] <= 0, 'uhours'] = np.nan
        df['hrwage'] = df['earnwke'] / df['uhours']
        dfs[year]['hrearnwt'] = dfs[year]['earnwt'] * dfs[year]['uhours']
    elif 'uhourse' in df.columns:
        # drop all values =< 0
        df.loc[df['uhourse'] <= 0, 'uhourse'] = np.nan
        df['hrwage'] = df['earnwke'] / df['uhourse']
        dfs[year]['hrearnwt'] = dfs[year]['earnwt'] * dfs[year]['uhourse']
    
    ## first adjust for inflation for each quarter ##
    # get the row from the inflation list
    inflation_row = inflation[j]
    realhrwage = df['hrwage'] 
    # if month between 1 and 3, then multiply by 100/33.804
    realhrwage = np.where(df['intmonth'] <= 3, realhrwage * 100/inflation_row[0], realhrwage)
    # if month between 4 and 6, then multiply by 100/34.728
    realhrwage = np.where((df['intmonth'] > 3) & (df['intmonth'] <= 6), realhrwage * 100/inflation_row[1], realhrwage)
    # if month between 7 and 9, then multiply by 100/35.590
    realhrwage = np.where((df['intmonth'] > 6) & (df['intmonth'] <= 9), realhrwage * 100/inflation_row[2], realhrwage)
    # if month between 10 and 12, then multiply by 100/36.451
    realhrwage = np.where((df['intmonth'] > 9) & (df['intmonth'] <= 12), realhrwage * 100/inflation_row[3], realhrwage)
    
    df['realhrwage'] = realhrwage
    # remove values < 2 and > 250
    df.loc[df['realhrwage'] < 2, 'realhrwage'] = np.nan
    df.loc[df['realhrwage'] > 250, 'realhrwage'] = np.nan


    ## get educ; treat gradeat vs grade92 separately ##
    # if grade32 exists, then need to convert that; otheerwise have to deal with gradeat and gradecp
    # create educ variable
    if 'grade92' in df.columns:
        df['educ'] = df['grade92']
        
        df.loc[df['grade92'] == 32, 'educ'] = 2.5
        df.loc[df['grade92'] == 33, 'educ'] = 5.5
        df.loc[df['grade92'] == 34, 'educ'] = 7.5
        df.loc[df['grade92'] == 35, 'educ'] = 9
        df.loc[df['grade92'] == 36, 'educ'] = 10
        df.loc[df['grade92'] == 37, 'educ'] = 11
        df.loc[df['grade92'] == 38, 'educ'] = 12
        df.loc[df['grade92'] == 39, 'educ'] = 12
        df.loc[df['grade92'] == 40, 'educ'] = 13
        df.loc[df['grade92'] == 41, 'educ'] = 14
        df.loc[df['grade92'] == 42, 'educ'] = 14
        df.loc[df['grade92'] == 43, 'educ'] = 16
        df.loc[df['grade92'] == 44, 'educ'] = 18
        df.loc[df['grade92'] == 45, 'educ'] = 18
        df.loc[df['grade92'] == 46, 'educ'] = 18

    else:
        df['educ'] = df['gradeat']
        # if gradecp is 0, then subtract 1
        df.loc[df['gradecp'] == 0, 'educ'] = df['educ'] - 1
    

    logwage = dfs[year]['realhrwage'].to_numpy()
    # conver to float
    logwage = logwage.astype(float)
    df['logwage'] = logwage
    df['lths'] = dfs[year]['educ'] < 12
    df['college'] = dfs[year]['educ'] >= 16
    df['exp'] = dfs[year]['age'] - dfs[year]['educ'] - 6
    df['exp2'] = dfs[year]['exp']**2

    # reset key value
    dfs[year] = df

/var/folders/zx/ck7rkn5j0_zfqqnb92k3668r0000gn/T/ipykernel_48653/1406542571.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['grade92'] == 32, 'educ'] = 2.5


In [64]:
summarize(dfs[19]['educ'])

Num observations:  291390
Mean:  13.625352620199733
Median:  13.0
Standard Deviation:  2.8183868628745694
Variance:  7.9433045088239576
1st percentile:  5.5
5th percentile:  10.0
10th percentile:  11.0
25th percentile:  12.0
50th percentile:  13.0
75th percentile:  16.0
90th percentile:  18.0
95th percentile:  18.0
99th percentile:  18.0
Skewness:  0.2835672619903265
Kurtosis:  4.374346395968131
Min:  2.5
Max:  31.0
